In [3]:
import os
import train
import utils
import numpy as np
import pandas as pd

from flask import Flask, jsonify, request
from tensorflow.keras.models import load_model
app = Flask(__name__)

model_path = os.path.join(os.getcwd(), 'model/saved_model.h5')
if os.path.exists(model_path):
    model = load_model(model_path)
else:
    model,_ = train.train(epoch=1)

@app.route('/', methods=['GET', 'POST'])
def predict():
    data = request.get_json()
    name = data.get('name')
    try:
        df = pd.DataFrame({'name': name})
    except:
        name = [name]
        df = pd.DataFrame({'name': name}, index=[0])
    df = utils.preprocess(df, train=False)
    result = model.predict(np.asarray(df['name'].values.tolist())).squeeze(axis=1)
    df['gender'] = ['Male' if logit > 0.5 else 'Female' for logit in result]
    df['probability'] = [logit if logit > 0.5 else 1.0 - logit for logit in result]
    gender_prediction = dict(zip(name, df['gender'].to_list()))
    return jsonify(gender_prediction)

app.run()

1188/1188 [==============================] - 101s 80ms/step - loss: 0.4103 - accuracy: 0.8141 - val_loss: 0.3501 - val_accuracy: 0.8479 - lr: 0.0010
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Name: ['James', 'Sam']
1/1 [==============================] - 1s 769ms/step


127.0.0.1 - - [18/Aug/2022 02:59:03] "POST / HTTP/1.1" 200 -


curl -X POST -d "{\"name\": [\"James\", \"Sam\"]}" http://127.0.0.1:5000/ -H "Content-Type:application/json"